# DETR：End-to-End Object Detection with Transformers

DETR是Facebook AI研究院提出的端到端的CV模型，其抛弃了传统one/two-stage、anchor-based/free的目标检测套路，采用直接在图像中预测对象集合的方法大大简化了检测流程。

## 整体结构

整个网络的架构如下所示：首先使用了一个基于CNN的Backbone对输入图像进行特征提取，论文中使用的是ResNet系列，之后在输入Transformer Encoder之前进行Position Embedding。Encoder操作论文中进行6次，之后进入到Decoder阶段，这个时候需要手动构造一个learnable的object queries用于在Transformer解码阶段的注意力交互，最后将Decoder的输出用于FFN层的输入，线性层预测图像中对象的类别以及中心点坐标和bbox的宽和高，论文中的预测head最后总是会生成100个预测对象。

但是在这一整个过程中DETR还是有区别于整个传统检测范式的地方，例如其最后的预测步骤没有例如NMS等后处理步骤，而是直接生成预测框。另外这篇论文中关于如何使用二分法进行预测框和bbox的匹配问题从而构建独特的损失函数也进行了详细描述。

<div align = "center">
    <img src = ./images/DETR-0.jpg width = 80%>
</div>

[参考博客文章](https://aitechtogether.com/python/94190.html)，可知DETR有以下特点：
- 端到端：去除了NMS和Anchor，没有那么多的超参数，计算量的大大减少，整个网络变得很简单；
- 基于Transformer：首次将Transformer引入到目标检测任务当中；
- 提出一种新的基于集合的损失函数：通过二分图匹配的方法强制模型输出一组独一无二的预测框，每个物体只会产生一个预测框，这样就讲目标检测问题之间转换为集合预测问题，所以才不用NMS，达到端到端的效果；
- 而且在decoder输入一组可学习的object query和encoder输出的共享全局特征，直接以并行方式强制最终输出的100个预测框，替代了anchor；

缺点：
- 对大物体的检测效果很好，但是对小物体的监测效果不是很好；训练起来比较慢；
- 由于query的设计以及初始化等问题，DETR模型从零开始训练需要超长的训练时间；

优点：
- 在COCO数据集上速度和精度和Faster RCNN差不多；可以扩展到很多细分的任务中，比如分割、追踪、多模态等；


下面实现一段简洁的DETR，作者也在论文中说明DETR并没有包含任何独特的网络结构，只需要框架中包含Backbone提取网络以及Transformer结构即可复现。

In [ ]:
import torch
from torch import nn
from torchvision.models import resnet50

class DETR(nn.Module):
    def __init__(self, num_classes, hidden_dim, nheads, num_encoder_layers, num_decoder_layers):
        super(DETR, self).__init__()
        # using ResNet-50 as the backbone
        self.backbone = nn.Sequential(*list(resnet50(pretrained=True).children())[:-2])
        self.conv = nn.Conv2d(2048, hidden_dim, 1)
        self.transformer = nn.Transformer(hidden_dim, nheads, num_encoder_layers, num_decoder_layers)
        
        self.linear_class = nn.Linear(hidden_dim, num_classes + 1)
        self.linear_bbox = nn.Linear(hidden_dim, 4)
        self.query_pos = nn.Parameter(torch.rand(100, hidden_dim))
        self.row_embed = nn.Parameter(torch.rand(50, hidden_dim // 2))
        self.col_embed = nn.Parameter(torch.rand(50, hidden_dim // 2))

    def forward(self, x):
        x = self.backbone(x)
        h = self.conv(x)
        H, W = h.shape[-2:]
        pos = torch.cat([
            self.col_embed[:W].unsqueeze(0).repeat(H, 1, 1),
            self.row_embed[:H].unsqueeze(1).repeat(1, W, 1)
        ], dim = -1).flatten(0, 1).unsqueeze(1)
        h = self.transformer(pos + h.flatten(2).permute(2,0,1),
                             self.query_pos.unqueeze(1))
        return self.linear_class(h), self.linear_bbox(h).sigmoid()

if __name__ == "__main__":
    detr = DETR(num_classes = 91, hidden_dim = 256, nheads = 8, num_encoder_layers = 6, num_decoder_layers=6)
    detr.eval()
    x = torch.randn(1, 3, 800, 1200)
    logits, bboxes = detr(x)